In [1]:
!pip install dodfminer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.2/132.2 kB 593.0 kB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 315.7 kB/s eta 0:00:00MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 514.7 kB/s eta 0:00:000:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 495.4 kB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 703.8 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.8/535.8 kB 875.8 kB/s eta 0:00:00m eta 0:00:010:01:01
  

In [1]:
#pipeline imports
import pandas as pd
import nltk

#Elementos do pipeline precisam herdar de TransformerMixin, sendo o ultimo (mais da direita) o Modelo que herda de BaseEstimator
from sklearn.base import TransformerMixin, BaseEstimator

#Pipeline precisa ser do sklearn.pipeline
from sklearn.pipeline import Pipeline

#Modelo usado no DODFMiner
from sklearn_crfsuite import CRF

99999999999999999999999999999999999999999999999999
from tqdm import tqdm



In [2]:
df = pd.read_csv('DODFCorpus_contratos_licitacoes_v2.csv')

In [3]:
df.head(10)

,Unnamed: 0,Unnamed: 0.1,id_dodf,tipo_rel,id_rel,anotador_rel,tipo_ent,id_ent,anotador_ent,offset,length,texto,id_ato
0,0,0,12_22.11.2018,REL_AVISO_LICITACAO,R53,anotador_6043174,modalidade_licitacao,1,anotador_6043174,432851,17,PREGAO ELETRONICO,12_22.11.2018-R53
1,1,1,12_22.11.2018,REL_AVISO_LICITACAO,R53,anotador_6043174,numero_licitacao,2,anotador_6043174,432872,8,116/2018,12_22.11.2018-R53
2,2,2,12_22.11.2018,REL_AVISO_LICITACAO,R53,anotador_6043174,valor_estimado_contratacao,6,anotador_6043174,433432,10,"767.907,37",12_22.11.2018-R53
3,3,3,12_22.11.2018,REL_AVISO_LICITACAO,R53,anotador_6043174,data_abertura_licitacao,7,anotador_6043174,433599,10,20/11/2018,12_22.11.2018-R53
4,4,4,12_22.11.2018,REL_AVISO_LICITACAO,R53,anotador_6043174,processo_gdf,490,anotador_6043174,433640,22,00410-00024534/2017-65,12_22.11.2018-R53
5,5,5,12_22.11.2018,REL_AVISO_LICITACAO,R53,anotador_6043174,sistema_compras,8,anotador_6043174,433716,21,www.compras.df.gov.br,12_22.11.2018-R53
6,6,6,12_22.11.2018,REL_AVISO_LICITACAO,R53,anotador_6043174,nome_responsavel,3,anotador_6043174,433818,25,GERARDA DA SILVA CARVALHO,12_22.11.2018-R53
7,7,7,12_22.11.2018,REL_AVISO_LICITACAO,R53,anotador_6043174,tipo_objeto,540,anotador_6043174,432927,9,aquisicao,12_22.11.2018-R53
8,8,8,12_22.11.2018,REL_AVISO_LICITACAO,R53,anotador_6043174,objeto_licitacao,542,anotador_6043174,432889,516,Registro de Precos visando a eventual aquisica...,12_22.11.2018-R53
9,9,9,12_22.11.2018,REL_AVISO_LICITACAO,R53,anotador_6043174,AVISO_LICITACAO,545,anotador_6043174,432830,1023,AVISOS DE LICITACOES\nPREGAO ELETRONICO No 116...,12_22.11.2018-R53


In [4]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id_dodf', 'tipo_rel', 'id_rel',
       'anotador_rel', 'tipo_ent', 'id_ent', 'anotador_ent', 'offset',
       'length', 'texto', 'id_ato'],
      dtype='object')

In [5]:
list_atos = ['AVISO_ANUL_REV_LICITACAO', 'AVISO_LICITACAO', 'AVISO_SUSPENSAO_LICITACAO', 
             'EXTRATO_ADITAMENTO_CONTRATUAL', 'EXTRATO_CONTRATO', 'EXTRATO_CONVENIO']



In [6]:
sep = '-||-'
df_contrato = df.pivot_table(index=['id_ato'], columns=['tipo_ent'],values='texto', aggfunc=lambda x: sep.join(x))
df_contrato.rename({'EXTRATO_CONTRATO':'texto'}, axis=1, inplace=True)

In [12]:
df_contrato['texto'][0]

'EXTRATO DO CONTRATO DE PRESTACAO DE SERVICOS No 03/2019\nNOS TERMOS DO PADRAO CEB\nPROCESSO: 00145-00000647/2019-23; DAS PARTES: ADMINISTRACAO REGIONAL DO\nRECANTO DAS EMAS - RA-XV x CEB DISTRIBUICAO S.A - CEB; DO FUNDAMENTO\nLEGAL: Inexigibilidade de Licitacao com base no caput do artigo 25 da Lei 8.666/1993; DO\nOBJETO: Prestacao e utilizacao de servico publico de energia eletrica entre a distribuidora e o\nconsumidor, de acordo com o Projeto Basico que integra o presente termo. DO VALOR: O valor\nanual estimado do contrato e de R$ 133.242,81 (cento e trinta e tres mil, duzentos e quarenta e dois\nreais e oitenta e um centavos); DA DOTACAO ORCAMENTARIA: Unidade Orcamentaria: 09117;\nPrograma de Trabalho: 04.128.6001.8517.0055; Natureza da Despesa: 3.3.90.39; Fonte de Recursos:\n100 e 120; Nota de Empenho no 2019NE00204, no valor inicial de R$ 9.993,20 (nove mil, novecentos\ne noventa e tres reais e vinte centavos); DA VIGENCIA: O Contrato tera vigencia de 12 (doze) meses\na contar d

In [8]:
df_contrato = df_contrato[~df_contrato['texto'].isnull() ]

In [9]:
df_contrato

tipo_ent,AVISO_ANUL_REV_LICITACAO,AVISO_LICITACAO,AVISO_SUSPENSAO_LICITACAO,EXTRATO_ADITAMENTO_CONTRATUAL,texto,EXTRATO_CONVENIO,cnpj_entidade_contratada,cnpj_entidade_convenente,cnpj_orgao_concedente,cnpj_orgao_contratante,...,processo_gdf,programa_trabalho,sistema_compras,tipo_objeto,unidade_orcamentaria,valor_contrato,valor_convenio,valor_estimado_contratacao,vigencia_contrato,vigencia_convenio
id_ato,,,,,,,,,,,,,,,,,,,,,
10_11.10.2019-R1,NaN,NaN,NaN,NaN,EXTRATO DO CONTRATO DE PRESTACAO DE SERVICOS N...,NaN,NaN,NaN,NaN,NaN,...,00145-00000647/2019-23,04.128.6001.8517.0055,NaN,NaN,09117,"133.242,81",NaN,NaN,O Contrato tera vigencia de 12 (doze) meses\na...,NaN
10_11.10.2019-R10,NaN,NaN,NaN,NaN,EXTRATO DO CONTRATO No 17/2019\nExtrato do Con...,NaN,02.633.335/0001-72,NaN,NaN,NaN,...,00121.0000.0393/2019-21,04.122.6003.2396.0033-||-04122600385170104,NaN,NaN,NaN,"10.000,00-||-10.000,00",NaN,NaN,NaN,NaN
10_11.10.2019-R11,NaN,NaN,NaN,NaN,EXTRATO DE CONTRATO\nContrato de prestacao de ...,NaN,05.411.801/0001-63,NaN,NaN,NaN,...,00071-00001158/2018-55,NaN,NaN,NaN,NaN,"9.072,00",NaN,NaN,"12 (doze) meses,\ncontados a partir da assinatura",NaN
10_11.10.2019-R14,NaN,NaN,NaN,NaN,EXTRATO DO CONTRATO N 57/2019\nProcesso: 00054...,NaN,NaN,NaN,NaN,NaN,...,00054-00018200/2019-48,28845090300NR0053,NaN,NaN,170393/00001,"200.500,00",NaN,NaN,12 (doze) meses a contar da\ndata de sua assin...,NaN
10_11.10.2019-R15,NaN,NaN,NaN,NaN,EXTRATO DO CONTRATO DE PRESTACAO DE SERVICOS N...,NaN,NaN,NaN,NaN,NaN,...,00400-00020718/2019-55,14.122.6002.8517.7250,NaN,NaN,44101,"494.959,00",NaN,NaN,O contrato tera vigencia de 12 (doze) meses,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9_10.9.2018-R1,NaN,NaN,NaN,NaN,EXTRATO DE CONTRATO\nPROCESSO: 00390-00001500/...,NaN,NaN,NaN,NaN,NaN,...,00390-00001500/2018-31,15.122.6001.8517.0131,NaN,NaN,28101,"9.220,00",NaN,NaN,"O contrato tera vigencia de 12 (doze) meses, a...",NaN
9_10.9.2018-R2,NaN,NaN,NaN,NaN,EXTRATO DO CONTRATO PARA PRESTACAO DE SERVICOS...,NaN,00.546.952/0001-05,NaN,NaN,00.394.718/0001-00,...,00050-00025147/2018-63,14.421.62 11 . 2 7 2 7 . 0 0 0 1,NaN,NaN,24101,"12.600,00",NaN,NaN,A vigencia contratual sera de 24 (vinte e quat...,NaN
9_10.9.2018-R3,NaN,NaN,NaN,NaN,EXTRATO DO CONTRATO PARA AQUISICAO DE BENS PEL...,NaN,00.716.050/0001-70,NaN,NaN,00.394.718/0001-00,...,00050-00032679/2018-57,14.421.6211.2727.0001,NaN,NaN,24101,"27.292,00",NaN,NaN,"A vigencia contratual sera de ate 31/12/2018,\...",NaN


In [10]:
class Processing(TransformerMixin):
  def __init__(self):
    pass

  def tokenize(self, sentence):
    text = nltk.word_tokenize(sentence)
    return text

  def fit(self, X, y = None):
    return self

  def transform(self, X, y = None):
    transformed = []
    for x in X:
      tokens = self.tokenize(x)
      transformed.append(tokens)
    return transformed

In [11]:
class FeatureExtractor(TransformerMixin):
  def __init__(self):
    pass

  def get_features(self, sentence):
    sent_features = []
    for i in range(len(sentence)):
      word_feat = {
        # Palavra atual
        'word': sentence[i].lower(),
        'capital_letter': sentence[i][0].isupper(),
        'all_capital': sentence[i].isupper(),
        'isdigit': sentence[i].isdigit(),
        # Uma palavra antes
        'word_before': '' if i == 0 else sentence[i-1].lower(),
        'word_before_isdigit': '' if i == 0 else sentence[i-1].isdigit(),
        'word_before_isupper': '' if i == 0 else sentence[i-1].isupper(),
        'word_before_istitle': '' if i == 0 else sentence[i-1].istitle(),
        # Uma palavra depois
        'word_after': '' if i+1 >= len(sentence) else sentence[i+1].lower(),
        'word_after_isdigit': '' if i+1 >= len(sentence) else sentence[i+1].isdigit(),
        'word_after_isupper': '' if i+1 >= len(sentence) else sentence[i+1].isupper(),
        'word_after_istitle': '' if i+1 >= len(sentence) else sentence[i+1].istitle(),

        'BOS': i == 0,
        'EOS': i == len(sentence)-1
      }
      sent_features.append(word_feat)
    return sent_features

  def fit(self, X, y = None):
    return self

  def transform(self, X, y = None):
    transformed = []
    for x in X:
      features = self.get_features(x)
      transformed.append(features)
    return transformed

In [12]:
class Model(BaseEstimator):

    def __init__(self):
        self.crf = CRF(
          algorithm = 'lbfgs',
          c1=0.35,
          c2=0.2,
          max_iterations=60,
          all_possible_transitions=True
        )

    def transform(self, X, y = None):
      return X

    def fit(self, X, y):
        self.crf.fit(X, y)
        return self

    def predict(self, x):
        pred = self.crf.predict(x)
        return pred

In [13]:
pipeline_custom = Pipeline([('pre-processing', Processing()), 
                            ('feature-extraction', FeatureExtractor()), ('model', Model())])

In [14]:
DEFAULT_TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize
class Tokenizer(TransformerMixin, BaseEstimator):
    """ Class to apply tokenizer to pandas DataFrame.
    """
    def __init__(self, tokenizer=DEFAULT_TOKENIZER):
        self.tokenizer = tokenizer


    def __call__(self, X, **kw_params):
        return self.tokenizer(X, **kw_params)


    def fit(self, X, y=None, **fit_params):
        return self


    def transform(self, X, **kw_params):
        if not isinstance(X, pd.Series):
            print("[preprocess.Tokenizer.transform] TYPE:", type(X))
            print('X:::: ', X)
            X = pd.Series(X)
        return X.map(self)


from sklearn.base import BaseEstimator, TransformerMixin

class IOBifyer(TransformerMixin, BaseEstimator):


    @staticmethod
    def find_entity(row, token, ignore_idx=0,
        tokenizer=DEFAULT_TOKENIZER):
        # TODO: aceitar opção de offset, para não ter tennhum tipo de problema
        """Searches for named entities on columns, except by ignore_idx-columns.

        ignore_idx: int indicating which column has
                    the TEXT where the named entity were extracted from
        """
        for idx, column in enumerate(row.keys()):
            if idx == ignore_idx:
                continue                    
                
            
            if isinstance(row[column], str):
                if token == word_tokenize(row[column])[0]:
                    return column
                spt = row[column].split('-||-')
                if len(spt) > 0:
                    for spt_token in spt:
                        if token == word_tokenize(spt_token)[0]:
                            print(token, spt)
                            return column
                       

        return None


    @staticmethod
    def generate_IOB_labels(row, idx, tokenizer, dbg={}):
        """[summary]

        Args:
            row ([pd.Series]): [pandas series having act text and entities text]
            idx ([int]): [index such that `row[idx]` has the whole act]
            tokenizer ([Callable]): [function to use to tokenize `row[idx]`]
            dbg (dict, optional): [dictionay for debug purposes]. Defaults to {}.

        Returns:
            [Iter[Iter[str]]]: [matrix of IOB labels]
        """
        labels = []
        entity_started = False
        text = row.iloc[idx]
        for token in word_tokenize(text):                         # Itera sobre cada token da anotação do ato.
            if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
                entity = IOBifyer.find_entity(row, token, idx)                 # Busca o token atual no primeiro token de todos os campos do df.
                if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                    entity_started = True
                    token_index = 1
                    labels.append('B-' + entity)
                else:
                    labels.append('O')
            else:     # Caso uma entidade já tenha sido identificada
                if token_index < len(word_tokenize(row[entity])) and \
                    token == word_tokenize(row[entity])[token_index]:
                    # Checa se o próximo token pertence à entidade
                    # e se o tamanho da entidade chegou ao fim.
                    labels.append('I-' + entity)
                    # Se a entidade ainda possui tokens e a comparação foi bem
                    # sucedida adicione o label I.
                    token_index += 1
                    if token_index >= len(word_tokenize(row[entity])):
                        entity_started = False
                else:
                    # Se o token n for igual ou a entidade chegou ao fim.
                    entity_started = False
                    labels.append('O')
        if labels[0] != 'O':
            dbg['l'] = dbg.get('l', []) + [(row, idx)]
        
        return labels


    @staticmethod
    def dump_iob(tokens_mat, labels_mat, path='dump.txt',
                            sep=' X X ', sent_sep='\n',):
        """This method dumps the token matrix according its IOB labels.

        For debug purposes, a list of list of pairs (token, label) is returned.
        Args:
            tokens_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to tokens]
            labels_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to IOB labels]
            path (str, optional): [Path to dump text file]. Defaults to 'dump.txt'.
            sep (str, optional): [description]. Defaults to ' X X '.
            sent_sep (str, optional): [description]. Defaults to '\n'.

        Returns:
            [List[LIst[Tuple(str, str)]]]: [list of list of pairs (token, label), as dumped. For debug purposes.]
        """
        dbg_mat = []
        if isinstance(path, Path):
            path = path.as_posix()
        if '/' in path:
            os.makedirs('/'.join(path.split('/')[:-1]), exist_ok=True)

        with open(path, 'w') as fp:
            for tokens_lis, labels_lis in zip(tokens_mat, labels_mat):
                dbg_mat.append([])
                for token, label in zip(tokens_lis, labels_lis):
                    dbg_mat[-1].append((token, label))
                    fp.write(f"{token}{sep}{label}\n")
                fp.write(sent_sep)
        return dbg_mat


    def __init__(self, column='act_column',
        tokenizer=DEFAULT_TOKENIZER):
        self.column = column
        self.tokenizer = tokenizer
        self.dbg = {}


    def fit(self, X=None, y=None, **fit_params):
        return self


    def transform(self, df):
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"`df` expected to be a pd.DataFrame. Got {type(df)}")
        if df.empty:
            print("[core.preprocess]Warning: empty DataFrame. There won't be ioblabels.")
            return pd.Series()

        idx = self.column if isinstance(self.column, int) else  \
                df.columns.get_loc(self.column)
        labels_row = []
        
        for index, row in tqdm(df.iterrows()):
            try:
                labels_row.append(
                    IOBifyer.generate_IOB_labels(
                        row, idx, self.tokenizer, self.dbg
                    )
                )
            except Exception as e:
                print("problem iobifyin row:", row)
                raise e
        return pd.Series(labels_row)

In [15]:
iob = IOBifyer(column='texto')
r = iob.transform(df_contrato)

1it [00:00,  1.28it/s]

100 ['100', '120']
120 ['100', '120']
17/2019 ['17/2019', '17/2019']
17/2019 ['17/2019', '17/2019']


2it [00:01,  2.06it/s]

04.122.6003.2396.0033 ['04.122.6003.2396.0033', '04122600385170104']
33.9039 ['33.9039', '339039']
100 ['100', '100']
2019NE00469 ['2019NE00469', '2019NE00470']
10.000,00 ['10.000,00', '10.000,00']
04122600385170104 ['04.122.6003.2396.0033', '04122600385170104']
339039 ['33.9039', '339039']
100 ['100', '100']
2019NE00470 ['2019NE00469', '2019NE00470']
10.000,00 ['10.000,00', '10.000,00']


25it [00:08,  4.99it/s]

...

50it [00:15,  3.15it/s]

2018NE00463 ['2018NE00463', '2018NE00464']
2018NE00464 ['2018NE00463', '2018NE00464']
732014482 ['732014482', '100000000']
100000000 ['732014482', '100000000']


51it [00:17,  2.97it/s]

2018NE00514 ['2018NE00514', '2018NE00515']


KeyboardInterrupt: 

In [75]:
word_tokenize('oi-||-como vai?')

['oi-||-como', 'vai', '?']